In [1]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 9.2MB/s 


## Импорт библиотек

In [0]:
import string
import re
from pymorphy2 import MorphAnalyzer
import pprint
import pandas as pd
from pprint import pprint
from nltk.util import ngrams
from collections import Counter 
from nltk.corpus import stopwords 
import nltk.data
import numpy as np

In [156]:
! wget https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/Транскрибации/all_transcripts.xlsx
! wget https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/Golden%20standard/NDS.csv
! wget https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/Golden%20standard/Pic.csv
! wget https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/Golden%20standard/Sib.csv

--2020-01-29 11:53:02--  https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D0%BA%D1%80%D0%B8%D0%B1%D0%B0%D1%86%D0%B8%D0%B8/all_transcripts.xlsx
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘all_transcripts.xlsx.2’

all_transcripts.xls     [ <=>                ]  63.00K  --.-KB/s    in 0.1s    

2020-01-29 11:53:03 (500 KB/s) - ‘all_transcripts.xlsx.2’ saved [64513]

--2020-01-29 11:53:04--  https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/Golden%20standard/NDS.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘NDS.csv.1’

NDS.csv.1               [ <=>                ] 242.27K  --.-KB/s    in 0.06

In [91]:
df_all = pd.read_excel('all_transcripts.xlsx', encoding="utf-8")
df_all.head()

,audio_ID,alphacep_transcripts,abk_transcripts
0,NDS_001-f-z.eaf,NaN,азат аллё а потом вот опять заново уже а по по...
1,NDS_002-f-z.eaf,слуга прошла подальше от прошла галлия а этот ...,хорошо да да гулять а я я ульянова задачка у н...
2,NDS_003-f-z.eaf,NaN,например спасибо я назад одна с нами наименее ...
3,NDS_004-m-z.eaf,NaN,я я готова я не пошёл
4,NDS_005-m-z.eaf,NaN,говорите а да манакова


## Задаем корпуса

In [0]:
df_pic_alpha = df_all.loc[
       (df_all.audio_ID.str.contains('Pic'))&df_all.alphacep_transcripts.notnull(),
       ['audio_ID','alphacep_transcripts']
       ]

df_nds_alpha = df_all.loc[
       (df_all.audio_ID.str.contains('NDS'))&df_all.alphacep_transcripts.notnull(),
       ['audio_ID','alphacep_transcripts']
       ]

df_sib_alpha=df_all.loc[
       (df_all.audio_ID.str.contains('Sib'))&df_all.alphacep_transcripts.notnull(),
       ['audio_ID','alphacep_transcripts']
       ]

df_nds_abk = df_all.loc[
       (df_all.audio_ID.str.contains('NDS'))&df_all.abk_transcripts.notnull(),
       ['audio_ID','abk_transcripts']
       ]
df_pic_abk = df_all.loc[
       (df_all.audio_ID.str.contains('Pic'))&df_all.abk_transcripts.notnull(),
       ['audio_ID','abk_transcripts']
       ]
df_sib_abk = df_all.loc[
       (df_all.audio_ID.str.contains('Sib'))&df_all.abk_transcripts.notnull(),
       ['audio_ID','abk_transcripts']
       ]

Читаем файлы "Золотого стандарта"

In [0]:
df_nds_gs = pd.read_csv('NDS.csv', encoding="utf-8", sep=",")
df_pic_gs = pd.read_csv('Pic.csv', encoding="utf-8", sep=",")
df_sib_gs = pd.read_csv('Sib.csv', encoding="utf-8", sep=",")

## Объявляем функции

Функция переименования столбцов

In [0]:
'''
Функция переименовывает столбцы в корпусах для их унификации и дальнейшей обработки
Аргументы: corpus - датафрейм с корпусом
trnscrbr - принимает значения 'abk' и 'alpha' для уточнения подкорпуса транскрибаций
'''

def rename_col (corpus, trnscrbr):
    if trnscrbr == 'abk':
        corpus = corpus.rename(columns={'audio_ID':'ID','abk_transcripts':'Text'})
    else:
        corpus = corpus.rename(columns = {'audio_ID':'ID','alphacep_transcripts':'Text'})
    return corpus

Функция создания списка ID

In [0]:
def get_id_lst(corpus):
    return list(corpus['ID'])

Предобработка текста

In [0]:
#Предобрабатываем текст: убираем пунктуацию, приводим все к нижнему регистру и расщепляем на слова:

def normalize(text):
    text = re.sub(r'[\[\]\n“”—!"\',\.:;\?`―\\]', '', text)
    text = text.lower().split()
    return text

Подсчет метрик

In [0]:
#Функция, которая извлекает набор уникальных словоформ и подсчитывает их количество: 

def unique_wordforms (tokens):
    tokens = sorted(set(tokens))
    return len(tokens)

In [0]:
'''
Функция, которая подсчитывает метрики и сохраняет их в двумерный список
Переменные: 
df_corp - датафрейм с транскрибациями, содержащий две колонки: ID, Text
corp_id - список ID-шников
'''
def count_metrics_to_lst (df_corp, corp_id):  
    lst_corp = []
    for elem in corp_id:
        elem_corp = str(df_corp[df_corp['ID'] == elem]['Text'].values)
        elem_corp = normalize(elem_corp)
        length = len(elem_corp) # кол-во слов
        elem_corp_token = unique_wordforms(elem_corp) # кол-во уникальных словоформ
        lst_elem = [] 
        lst_elem.append(elem)
        lst_elem.append(length)
        lst_elem.append(elem_corp_token)
        lst_corp.append(lst_elem)

    return lst_corp

## Предобработка и подсчет метрик

Переименовываем столбцы

In [0]:
df_nds_abk = rename_col(df_nds_abk,"abk")
df_pic_alpha = rename_col(df_pic_alpha, "alpha")
df_nds_alpha = rename_col(df_nds_alpha, "alpha")
df_pic_abk = rename_col(df_pic_abk,"abk")
df_sib_abk = rename_col(df_sib_abk,"abk")
df_sib_alpha = rename_col(df_sib_alpha, "alpha")

Создаем списки айдишников

In [0]:
#корпуса альфы
corp_id_pic_alpha = get_id_lst(df_pic_alpha)
corp_id_nds_alpha = get_id_lst(df_nds_alpha)
corp_id_sib_alpha = get_id_lst(df_sib_alpha)
#корпуса абк
corp_id_pic_abk= get_id_lst(df_pic_abk)
corp_id_nds_abk= get_id_lst(df_nds_abk)
corp_id_sib_abk= get_id_lst(df_sib_abk)

Подсчет для альфа

In [134]:
df_pic_alpha

,ID,Text
35,Pic-RUS_01-f_Pr-R.eaf,жилбыл один дяденька по его жены скоро должно ...
38,Pic-RUS_01-f_Ski-T.eaf,генин жизни одного очень увлекающийся спортом ...
39,Pic-RUS_02-f_Pr-R.eaf,однозначным был день рождения мышь решил подар...
42,Pic-RUS_02-f_Ski-T.eaf,этот человек встал рано утром позавтракал а по...
45,Pic-RUS_03-m_Ski-R.eaf,знакомым мне здесь рассказали одну смешную и п...
46,Pic-RUS_03-m_Ski-T.eaf,один чувак решил покататься на лыжах както ран...
52,Pic-RUS_05-m_Pr-T.eaf,однажды константин решил подарить жене подарок...
54,Pic-RUS_05-m_Ski-T.eaf,миша проснулся очень рано гдето в восемь часов...
55,Pic-RUS_06-f_Pr-R.eaf,один мужчина решил подарить своей жене какойни...
58,Pic-RUS_06-f_Ski-T.eaf,жилбыл один дядечка один раз он проснулся утр...


In [0]:
lst=[]
lst += count_metrics_to_lst(df_pic_alpha,corp_id_pic_alpha)
lst += count_metrics_to_lst(df_nds_alpha, corp_id_nds_alpha)
lst += count_metrics_to_lst(df_sib_alpha, corp_id_sib_alpha)

In [0]:
df_alpha = pd.DataFrame(lst, columns=['ID', 'words_num_alpha','unique_num_alpha'])

In [137]:
df_alpha

,ID,words_num_alpha,unique_num_alpha
0,Pic-RUS_01-f_Pr-R.eaf,143,117
1,Pic-RUS_01-f_Ski-T.eaf,85,72
2,Pic-RUS_02-f_Pr-R.eaf,81,67
3,Pic-RUS_02-f_Ski-T.eaf,68,55
4,Pic-RUS_03-m_Ski-R.eaf,86,71
5,Pic-RUS_03-m_Ski-T.eaf,70,58
6,Pic-RUS_05-m_Pr-T.eaf,128,98
7,Pic-RUS_05-m_Ski-T.eaf,109,81
8,Pic-RUS_06-f_Pr-R.eaf,41,38
9,Pic-RUS_06-f_Ski-T.eaf,61,51


Подсчет для абк

In [0]:
lst2=[]
lst2 += count_metrics_to_lst(df_pic_abk, corp_id_pic_abk)
lst2 += count_metrics_to_lst(df_nds_abk, corp_id_nds_abk)
lst2 += count_metrics_to_lst(df_sib_abk, corp_id_sib_abk)

In [0]:
df_abk = pd.DataFrame(lst2, columns=['ID', 'words_num_abk','unique_num_abk'])

In [141]:
df_abk

,ID,words_num_abk,unique_num_abk
0,Pic-RUS_01-f_Pr-R.eaf,156,121
1,Pic-RUS_01-f_Pr-T.eaf,139,100
2,Pic-RUS_01-f_Ski-R.eaf,101,68
3,Pic-RUS_01-f_Ski-T.eaf,91,75
4,Pic-RUS_02-f_Pr-R.eaf,90,71
...,...,...,...
72,Sib_03-m.eaf,91,77
73,Sib_04-m.eaf,155,120
74,Sib_06-f.eaf,44,37
75,Sib_14-f.eaf,70,59


Подсчет метрик для золотого стандарта для сравнения с АБК



In [0]:
lst3=[]
lst3 += count_metrics_to_lst(df_pic_gs, corp_id_pic_abk)
lst3 += count_metrics_to_lst(df_nds_gs, corp_id_nds_abk)
lst3 += count_metrics_to_lst(df_sib_gs, corp_id_sib_abk)

In [0]:
df_gs_abk = pd.DataFrame(lst3, columns=['ID', 'words_num_gs_abk','unique_num_gs_abk'])

In [144]:
df_gs_abk

,ID,words_num_gs_abk,unique_num_gs_abk
0,Pic-RUS_01-f_Pr-R.eaf,164,124
1,Pic-RUS_01-f_Pr-T.eaf,152,108
2,Pic-RUS_01-f_Ski-R.eaf,109,74
3,Pic-RUS_01-f_Ski-T.eaf,92,76
4,Pic-RUS_02-f_Pr-R.eaf,114,77
...,...,...,...
72,Sib_03-m.eaf,165,111
73,Sib_04-m.eaf,191,116
74,Sib_06-f.eaf,103,67
75,Sib_14-f.eaf,84,68


Подсчет метрик ЗС в сравнении с альфой

In [0]:
lst4=[]
lst4 += count_metrics_to_lst(df_pic_gs,corp_id_pic_alpha)
lst4 += count_metrics_to_lst(df_nds_gs, corp_id_nds_alpha)
lst4 += count_metrics_to_lst(df_sib_gs, corp_id_sib_alpha)

In [0]:
df_gs_alpha = pd.DataFrame(lst, columns=['ID', 'words_num_gs_alpha','unique_num_gs_alpha'])

In [148]:
df_gs_alpha

,ID,words_num_gs_alpha,unique_num_gs_alpha
0,Pic-RUS_01-f_Pr-R.eaf,164,124
1,Pic-RUS_01-f_Ski-T.eaf,92,76
2,Pic-RUS_02-f_Pr-R.eaf,114,77
3,Pic-RUS_02-f_Ski-T.eaf,80,64
4,Pic-RUS_03-m_Ski-R.eaf,98,75
5,Pic-RUS_03-m_Ski-T.eaf,73,61
6,Pic-RUS_05-m_Pr-T.eaf,133,98
7,Pic-RUS_05-m_Ski-T.eaf,110,84
8,Pic-RUS_06-f_Pr-R.eaf,50,44
9,Pic-RUS_06-f_Ski-T.eaf,68,55


Объединяем датафреймы

In [0]:
df_abk_merged = pd.merge(df_gs_abk,df_abk, how='outer')

In [150]:
df_abk_merged

,ID,words_num_gs_abk,unique_num_gs_abk,words_num_abk,unique_num_abk
0,Pic-RUS_01-f_Pr-R.eaf,164,124,156,121
1,Pic-RUS_01-f_Pr-T.eaf,152,108,139,100
2,Pic-RUS_01-f_Ski-R.eaf,109,74,101,68
3,Pic-RUS_01-f_Ski-T.eaf,92,76,91,75
4,Pic-RUS_02-f_Pr-R.eaf,114,77,90,71
...,...,...,...,...,...
72,Sib_03-m.eaf,165,111,91,77
73,Sib_04-m.eaf,191,116,155,120
74,Sib_06-f.eaf,103,67,44,37
75,Sib_14-f.eaf,84,68,70,59


In [0]:
df_alpha_merged = pd.merge(df_gs_alpha,df_alpha, how='outer')

In [153]:
df_alpha_merged

,ID,words_num_gs_alpha,unique_num_gs_alpha,words_num_alpha,unique_num_alpha
0,Pic-RUS_01-f_Pr-R.eaf,164,124,143,117
1,Pic-RUS_01-f_Ski-T.eaf,92,76,85,72
2,Pic-RUS_02-f_Pr-R.eaf,114,77,81,67
3,Pic-RUS_02-f_Ski-T.eaf,80,64,68,55
4,Pic-RUS_03-m_Ski-R.eaf,98,75,86,71
5,Pic-RUS_03-m_Ski-T.eaf,73,61,70,58
6,Pic-RUS_05-m_Pr-T.eaf,133,98,128,98
7,Pic-RUS_05-m_Ski-T.eaf,110,84,109,81
8,Pic-RUS_06-f_Pr-R.eaf,50,44,41,38
9,Pic-RUS_06-f_Ski-T.eaf,68,55,61,51


Выгружаем в .xlsx файл

In [0]:
df_abk_merged.to_excel("abk_count_merged.xlsx")

In [0]:
df_alpha_merged.to_excel("count_alpha_merged.xlsx")